In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
sentence = "The quick brown fox jumps over the lazy dog"

## Simple tokenization by splitting on spaces, ideally more complex tokenization would be used like BPE or WordPiece
sentence = sentence.split()
n = len(sentence)

print(f"Tokenized sentence: {sentence}")
print(f"Number of tokens: {len(sentence)}")

Tokenized sentence: ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
Number of tokens: 9


In [4]:
## Sample word embeddings, ideally these would be learned in the language modelling process or loaded from a pre-trained model like GloVe or Word2Vec

# Shape of embeddings: (n, d) where n is number of tokens and d is embedding dimension
embeddings = torch.tensor([
        [1.0, 0.5, 0.2, 0.8], 
        [0.3, 1.0, 0.7, 0.1],  
        [0.6, 0.2, 1.0, 0.4],  
        [0.9, 0.8, 0.3, 1.0],  
        [0.4, 0.6, 0.8, 0.2],  
        [0.7, 0.3, 0.5, 0.9],  
        [1.0, 0.5, 0.2, 0.8],  
        [0.2, 0.9, 0.4, 0.6],  
        [0.8, 0.4, 0.9, 0.3]  
    ])

print("Word embeddings (4-dimensional):")
for i, word in enumerate(sentence):
    print(f"  {word:4}: {embeddings[i]}")
print()

Word embeddings (4-dimensional):
  The : tensor([1.0000, 0.5000, 0.2000, 0.8000])
  quick: tensor([0.3000, 1.0000, 0.7000, 0.1000])
  brown: tensor([0.6000, 0.2000, 1.0000, 0.4000])
  fox : tensor([0.9000, 0.8000, 0.3000, 1.0000])
  jumps: tensor([0.4000, 0.6000, 0.8000, 0.2000])
  over: tensor([0.7000, 0.3000, 0.5000, 0.9000])
  the : tensor([1.0000, 0.5000, 0.2000, 0.8000])
  lazy: tensor([0.2000, 0.9000, 0.4000, 0.6000])
  dog : tensor([0.8000, 0.4000, 0.9000, 0.3000])



In [5]:
## Sample positional encodings, typically these would be generated using math functions or learned during training or RoPE

positional_embeddings = torch.tensor([
    [0.0, 1.0, 0.0, 1.0],  
    [0.1, 0.9, 0.1, 0.9],  
    [0.2, 0.8, 0.2, 0.8],  
    [0.3, 0.7, 0.3, 0.7],  
    [0.4, 0.6, 0.4, 0.6],  
    [0.5, 0.5, 0.5, 0.5],  
    [0.6, 0.4, 0.6, 0.4],  
    [0.7, 0.3, 0.7, 0.3],  
    [0.8, 0.2, 0.8, 0.2]   
])

print("Positional embeddings (Same dimesnions as word embeddings):")
for i, word in enumerate(sentence):
    print(f"  Pos {i} ({word}): {positional_embeddings[i]}")
print()

Positional embeddings (Same dimesnions as word embeddings):
  Pos 0 (The): tensor([0., 1., 0., 1.])
  Pos 1 (quick): tensor([0.1000, 0.9000, 0.1000, 0.9000])
  Pos 2 (brown): tensor([0.2000, 0.8000, 0.2000, 0.8000])
  Pos 3 (fox): tensor([0.3000, 0.7000, 0.3000, 0.7000])
  Pos 4 (jumps): tensor([0.4000, 0.6000, 0.4000, 0.6000])
  Pos 5 (over): tensor([0.5000, 0.5000, 0.5000, 0.5000])
  Pos 6 (the): tensor([0.6000, 0.4000, 0.6000, 0.4000])
  Pos 7 (lazy): tensor([0.7000, 0.3000, 0.7000, 0.3000])
  Pos 8 (dog): tensor([0.8000, 0.2000, 0.8000, 0.2000])



In [6]:
## The final input to the Attention block is the sum of the word embeddings and positional encodings

input_embeddings = embeddings + positional_embeddings

print("Input embeddings (word + positional):")
for i, word in enumerate(sentence):
    print(f"  {word:4}: {input_embeddings[i]}")
print()

Input embeddings (word + positional):
  The : tensor([1.0000, 1.5000, 0.2000, 1.8000])
  quick: tensor([0.4000, 1.9000, 0.8000, 1.0000])
  brown: tensor([0.8000, 1.0000, 1.2000, 1.2000])
  fox : tensor([1.2000, 1.5000, 0.6000, 1.7000])
  jumps: tensor([0.8000, 1.2000, 1.2000, 0.8000])
  over: tensor([1.2000, 0.8000, 1.0000, 1.4000])
  the : tensor([1.6000, 0.9000, 0.8000, 1.2000])
  lazy: tensor([0.9000, 1.2000, 1.1000, 0.9000])
  dog : tensor([1.6000, 0.6000, 1.7000, 0.5000])



In [7]:
d_model = input_embeddings.shape[1]  # Embedding dimension
d_k = 3 # Dimension of keys and queries (generally kept smaller to make Q, K and V matrices low rank for efficiency)

In [8]:
torch.manual_seed(42)  # For reproducible results

## Shape of the Q, K and V matrices is d x d_k and for the output projection matrix is d_k x d to project the attention output back to d dimensions
W_q = torch.randn(d_model, d_k, dtype=torch.float32) * 0.3  
W_k = torch.randn(d_model, d_k, dtype=torch.float32) * 0.3  
W_v = torch.randn(d_model, d_k, dtype=torch.float32) * 0.3  
W_o = torch.randn(d_k, d_model, dtype=torch.float32) * 0.3 

print(f"W_q (Query weights) shape: {W_q.shape}")
print(W_q)
print(f"\nW_k (Key weights) shape: {W_k.shape}")
print(W_k)
print(f"\nW_v (Value weights) shape: {W_v.shape}")
print(W_v)
print()

print(f"W_o (Output projection weights) shape: {W_o.shape}")
print(W_o)

W_q (Query weights) shape: torch.Size([4, 3])
tensor([[ 0.1010,  0.0386,  0.0703],
        [ 0.0691, -0.3369, -0.0559],
        [ 0.6625, -0.1914,  0.1385],
        [ 0.0802,  0.1605,  0.2428]])

W_k (Key weights) shape: torch.Size([4, 3])
tensor([[ 0.3331, -0.5069, -0.2967],
        [ 0.2874,  0.3966,  0.2452],
        [-0.2298, -0.2252,  0.4058],
        [ 0.2059, -0.0983,  0.2385]])

W_v (Value weights) shape: torch.Size([4, 3])
tensor([[ 0.0845,  0.0168,  0.1568],
        [-0.0715, -0.0150,  0.1579],
        [-0.0025,  0.2187,  0.0399],
        [ 0.2592, -0.3047, -0.2666]])

W_o (Output projection weights) shape: torch.Size([3, 4])
tensor([[ 0.0449, -0.0627, -0.1161,  0.2974],
        [ 0.1404, -0.0615, -0.2223,  0.1086],
        [ 0.5760, -0.0676, -0.1025,  0.0912]])


In [9]:
Q = torch.matmul(input_embeddings, W_q) # Shape: (n, d_k)  
K = torch.matmul(input_embeddings, W_k) # Shape: (n, d_k)
V = torch.matmul(input_embeddings, W_v) # Shape: (n, d_k)

print(f"\nQ (Queries) shape: {Q.shape}")
print(Q)
print(f"\nK (Keys) shape: {K.shape}")
print(K)
print(f"\nV (Values) shape: {V.shape}")
print(V)
print()


Q (Queries) shape: torch.Size([9, 3])
tensor([[ 0.4815, -0.2161,  0.4512],
        [ 0.7819, -0.6172,  0.2755],
        [ 1.0411, -0.3431,  0.4579],
        [ 0.7587, -0.3010,  0.4964],
        [ 1.0228, -0.4746,  0.3496],
        [ 0.9512, -0.1899,  0.5181],
        [ 0.8500, -0.2019,  0.4644],
        [ 0.9747, -0.4356,  0.3671],
        [ 1.3694, -0.3854,  0.4359]])

K (Keys) shape: torch.Size([9, 3])
tensor([[ 1.0888, -0.1340,  0.5815],
        [ 0.7014,  0.2724,  0.9102],
        [ 0.5252, -0.3971,  0.7809],
        [ 1.0430, -0.3156,  0.6606],
        [ 0.5004, -0.2785,  0.7345],
        [ 0.6881, -0.6539,  0.5798],
        [ 0.8549, -0.7523,  0.3567],
        [ 0.5772, -0.3165,  0.6882],
        [ 0.4177, -1.0051,  0.4814]])

V (Values) shape: torch.Size([9, 3])
tensor([[ 0.4432, -0.5103, -0.0783],
        [ 0.1551, -0.1514,  0.1281],
        [ 0.3040, -0.1047,  0.0113],
        [ 0.4332, -0.3890, -0.0043],
        [ 0.1861,  0.0142,  0.1496],
        [ 0.4045, -0.1996, -0.0188

In [ ]:
## Creating the mask for causal attention where each token can only attend to itself and previous tokens, where there is a 1 in the mask matrix, the attention score is kept, where there is a 0 the attention score is masked out

mask = torch.tril(torch.ones(n, n))  # Shape: (n, n)
print(f"Mask shape: {mask.shape}")
print(mask)

Mask shape: torch.Size([9, 9])
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [14]:
attention_scores = torch.matmul(Q, K.T)  # Shape: (n, n)

print(f"Attention scores shape: {attention_scores.shape}")
print(attention_scores)

Attention scores shape: torch.Size([9, 9])
tensor([[0.8156, 0.6896, 0.6911, 0.8685, 0.6326, 0.7342, 0.7352, 0.6569, 0.6356],
        [1.0943, 0.6311, 0.8709, 1.1923, 0.7655, 1.1013, 1.2310, 0.8363, 1.0796],
        [1.4458, 1.0536, 1.0407, 1.4966, 0.9528, 1.2062, 1.3114, 1.0247, 1.0002],
        [1.1551, 0.9020, 0.9057, 1.2142, 0.8281, 1.0067, 1.0521, 0.8748, 0.8584],
        [1.3806, 0.9064, 0.9988, 1.4476, 0.9008, 1.2169, 1.3562, 0.9812, 1.0727],
        [1.3625, 1.0871, 0.9796, 1.3943, 0.9094, 1.0791, 1.1408, 0.9657, 0.8376],
        [1.2226, 0.9639, 0.8893, 1.2570, 0.8227, 0.9862, 1.0442, 0.8741, 0.7816],
        [1.3331, 0.8992, 0.9716, 1.3966, 0.8786, 1.1683, 1.2919, 0.9531, 1.0217],
        [1.7961, 1.2522, 1.2127, 1.8378, 1.1126, 1.4470, 1.6160, 1.2123, 1.1693]])


In [15]:
masked_attention_scores = attention_scores.clone()

## Wherever the mask is 0, set the attention score to -inf so that after softmax, the attention weights becomes 0 for the future tokens
masked_attention_scores = masked_attention_scores.masked_fill(mask == 0, float('-inf'))

print(f"\nMasked attention scores shape: {masked_attention_scores.shape}")
print(masked_attention_scores)


Masked attention scores shape: torch.Size([9, 9])
tensor([[0.8156,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf],
        [1.0943, 0.6311,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf],
        [1.4458, 1.0536, 1.0407,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf],
        [1.1551, 0.9020, 0.9057, 1.2142,   -inf,   -inf,   -inf,   -inf,   -inf],
        [1.3806, 0.9064, 0.9988, 1.4476, 0.9008,   -inf,   -inf,   -inf,   -inf],
        [1.3625, 1.0871, 0.9796, 1.3943, 0.9094, 1.0791,   -inf,   -inf,   -inf],
        [1.2226, 0.9639, 0.8893, 1.2570, 0.8227, 0.9862, 1.0442,   -inf,   -inf],
        [1.3331, 0.8992, 0.9716, 1.3966, 0.8786, 1.1683, 1.2919, 0.9531,   -inf],
        [1.7961, 1.2522, 1.2127, 1.8378, 1.1126, 1.4470, 1.6160, 1.2123, 1.1693]])


In [16]:
masked_attention_scores /= torch.sqrt(torch.tensor(d_k, dtype=torch.float32)) # Scale by sqrt(d_k)

print(f"\nScaled masked attention scores shape: {masked_attention_scores.shape}")
print(masked_attention_scores)


Scaled masked attention scores shape: torch.Size([9, 9])
tensor([[0.4709,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.6318, 0.3643,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.8348, 0.6083, 0.6008,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.6669, 0.5208, 0.5229, 0.7010,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.7971, 0.5233, 0.5766, 0.8358, 0.5201,   -inf,   -inf,   -inf,   -inf],
        [0.7866, 0.6276, 0.5656, 0.8050, 0.5250, 0.6230,   -inf,   -inf,   -inf],
        [0.7059, 0.5565, 0.5134, 0.7258, 0.4750, 0.5694, 0.6029,   -inf,   -inf],
        [0.7697, 0.5191, 0.5610, 0.8063, 0.5073, 0.6745, 0.7459, 0.5503,   -inf],
        [1.0370, 0.7229, 0.7001, 1.0610, 0.6424, 0.8354, 0.9330, 0.6999, 0.6751]])


In [17]:
attention_weights = F.softmax(masked_attention_scores, dim=-1)

print(f"\nAttention weights shape: {attention_weights.shape}")
print(attention_weights)


Attention weights shape: torch.Size([9, 9])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5665, 0.4335, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3863, 0.3080, 0.3057, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2656, 0.2295, 0.2300, 0.2749, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2294, 0.1744, 0.1840, 0.2384, 0.1739, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1890, 0.1612, 0.1515, 0.1925, 0.1455, 0.1604, 0.0000, 0.0000, 0.0000],
        [0.1594, 0.1373, 0.1315, 0.1626, 0.1265, 0.1390, 0.1438, 0.0000, 0.0000],
        [0.1411, 0.1099, 0.1146, 0.1464, 0.1086, 0.1283, 0.1378, 0.1133, 0.0000],
        [0.1375, 0.1005, 0.0982, 0.1409, 0.0927, 0.1124, 0.1239, 0.0982, 0.0958]])


In [18]:
output = torch.matmul(attention_weights, V)  # Shape: (n, d_k)

print(f"\nAttention output shape: {output.shape}")
print(output)


Attention output shape: torch.Size([9, 3])
tensor([[ 0.4432, -0.5103, -0.0783],
        [ 0.3183, -0.3547,  0.0112],
        [ 0.3119, -0.2758,  0.0127],
        [ 0.3423, -0.3013,  0.0100],
        [ 0.3203, -0.2530,  0.0315],
        [ 0.3301, -0.2415,  0.0255],
        [ 0.3367, -0.2306,  0.0373],
        [ 0.3273, -0.2102,  0.0472],
        [ 0.3197, -0.1744,  0.0674]])


In [19]:
final_output = torch.matmul(output, W_o) # Shape: (n, d)

## Projecting the output back to d dimensions
print(f"\nFinal output shape: {final_output.shape}")
print(final_output)


Final output shape: torch.Size([9, 4])
tensor([[-0.0968,  0.0089,  0.0700,  0.0693],
        [-0.0290,  0.0011,  0.0407,  0.0572],
        [-0.0174, -0.0035,  0.0238,  0.0640],
        [-0.0211, -0.0036,  0.0262,  0.0700],
        [-0.0030, -0.0066,  0.0158,  0.0706],
        [-0.0044, -0.0076,  0.0127,  0.0743],
        [ 0.0043, -0.0094,  0.0083,  0.0785],
        [ 0.0124, -0.0108,  0.0039,  0.0788],
        [ 0.0287, -0.0139, -0.0053,  0.0823]])
